<h4>Makes Jupyter Notebook Wider</h4>

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<h1>Imports</h1>

In [5]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

import pdb

<h1>Global Variables</h1>

In [6]:
background = None

accumulated_weight = 0.5

top = 20
bottom = 300
right = 300
left = 600

<h1>Functions</h1>

<h4>Find the Average</h4>
<p>The Function calculates the weighted sum of the source image in order to distinguish the hand from the background</p>

In [7]:
def cumulative_avg(frame, accumulated_weight):
    """
    Given a frame and a previous accumulated weight, computed the weighted average of the image passed in.
    """
    global background
    #creates a new background frame for the first time
    if background is None:
        background = frame.copy().astype('float')
        return None
    #compute wieghted average, accumlate it and update background
    cv2.accumulateWeighted(frame, background, accumulated_weight)


<h4>Find your Hand</h4>
<p>Distinguish hand from background</p>

In [8]:
def segment(frame, thresh_min=25):
    global background
    
    # Calculates the Absolute Differentce between the backgroud and the passed in frame
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    # Apply a threshold to the image so we can grab the foreground
    # We only need the threshold, so we will throw away the first item in the tuple with an underscore
    ret, thresh = cv2.threshold(diff,thresh_min,255,cv2.THRESH_BINARY)
    
    #Grab the external contours form the image
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # If length of contours list is 0, then we didn't grab any contours!
    if len(contours) == 0:
        return None
    else:
        #Assumes the largest object in the contour is the hand
        hand_segment = max(contours, key=cv2.contourArea)
        return (thresh, hand_segment)
    

<img src='Finger Count 10_19_2020 1_45_32 AM.png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>

<h4>Count Fingers</h4>

In [9]:
def count_fingers(thresh, hand):
    
    #calculated the convex hull of the hand segment
    convex = cv2.convexHull(hand)
    
    #Points of the fingers
    top    = tuple(convex[convex[:, :, 1].argmin()][0])
    bottom = tuple(convex[convex[:, :, 1].argmax()][0])
    left   = tuple(convex[convex[:, :, 0].argmin()][0])
    right  = tuple(convex[convex[:, :, 0].argmax()][0])
    
    # In theory, the center of the hand is half way between the top and bottom and halfway between left and right
    # horizontal center of hand
    center_x = (left[0]+right[0])//2
    #vertical center of hand
    center_y = (top[1]+bottom[1])//2

    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    
    # Calculate the Euclidean Distance between the center of the hand and the left, right, top, and bottom.
    distance = pairwise.euclidean_distances([(center_x, center_y)], Y=[left, right, top, bottom])[0]
   
    # Grab the largest distance
    max_distance = distance.max()
    
    # Create a circle with 90% radius of the max euclidean distance
    radius = int(0.8*max_distance)
    circumfrence = (2*np.pi*radius)
    
    # Grab ROI (Region of Interest)
    circular = np.zeros(thresh.shape[:2], dtype='uint8')
     # draw the circular ROI
    cv2.circle(circular,(center_x,center_y), radius, 255,10)
    
    # Using bit-wise AND with the cirle ROI as a mask.
    # This then returns the cut out obtained using the mask on the thresholded hand image.
    circular = cv2.bitwise_and(thresh, thresh, mask=circular)
    
    contours,hiearchy = cv2.findContours(circular.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # Finger count starts at 0      
    finger_count = 0
    
    # loop through the contours to see if we count any more fingers.
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        outside_wrist = ((center_y+(center_y*0.25)) > (y+h))
        limit_points = ((circumfrence*0.25) > cnt.shape[0]) 
        if outside_wrist and limit_points:
            finger_count += 1
    return finger_count

<h4>Run Program</h4>

In [23]:
cam = cv2.VideoCapture(0)
num_frames = 0

while True:
    ret, frame = cam.read()
    
    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
    if frame is None:
        break
    #clone frame
    frame_copy = frame.copy()
    #grab the region of interest
    roi = frame[top:bottom,right:left]
    
    #Apply grayscale and blue the ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # For the first 60 frames we will calculate the average of the background.
    # We will tell the user while this is happening
    if num_frames < 60:
        cumulative_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, 'WAIT! GETTING BACKGROUND', (200,400), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2)
            cv2.imshow('Finger Count', frame_copy)
    else:
        hand = segment(gray)
        if hand is not None:
            thresh, hand_segment = hand
            #Draws contours around real hand in live stream
            cv2.drawContours(frame_copy, [hand_segment,right,top],0,(255,0,0), 5)
            fingers = count_fingers(thresh, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Threshold',thresh)
    cv2.rectangle(frame_copy,(left,top), (right, bottom), (0,0,255), 5)
    num_frames += 1
    cv2.imshow('Finger Count', frame_copy)
    
    # Close windows with Esc
    control = cv2.waitKey(1)
    if control == 27 or control == 'q':
        break
cam.release()
cv2.destroyAllWindows()


<img src='Screenshot (55).png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>
<img src='Screenshot (56).png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>
<img src='Screenshot (57).png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>
<img src='Screenshot (58).png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>
<img src='Screenshot (60).png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>
<img src='Screenshot (61).png' alt='Markdown Finger Icon' style='float:left; margin-right: 10px;'/>